<a href="https://colab.research.google.com/github/aarsanjani/meansquares/blob/master/version2/prepare_data/Load_Social_distancing_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal

This colab loads the social distancing metric data from descarteslabs ( Social distancing research paper)

## Import packages

In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=5ea4f76b7c5a02618851685fc7ed8de2795164262e9f429a9c5887751583296d
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import wget
from pandas import Series, datetime
from pandas.plotting import scatter_matrix, autocorrelation_plot
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
import random
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from xgboost import XGBClassifier
from sklearn.mixture import GaussianMixture
from datetime import datetime

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Mount drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load data

In [4]:
socialdistancing_url = 'https://raw.githubusercontent.com/descarteslabs/DL-COVID-19/master/DL-us-mobility-daterow.csv'

socialdistancing_file = wget.download(socialdistancing_url)
print("filename : ",socialdistancing_file)
socialdistancing_data = pd.read_csv(socialdistancing_file,low_memory=False)
socialdistancing_data.tail(2)

filename :  DL-us-mobility-daterow.csv


,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
818805,2021-01-06,US,2,Wyoming,Uinta County,56041.0,782,3.340,106
818806,2021-01-07,US,2,Wyoming,Uinta County,56041.0,725,2.721,86


In [5]:
print(socialdistancing_data.date.min())
socialdistancing_data.date.max()

2020-03-01


'2021-01-07'

## Note (to another developers)

Filter data as per state and load to other data

# Filter for Texas state

In [7]:
texas_socialdistancing_data = socialdistancing_data[socialdistancing_data['admin1'] == 'Texas']
texas_socialdistancing_data.head()

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
656088,2020-03-01,US,1,Texas,NaN,48.0,633045,6.619,72
656089,2020-03-02,US,1,Texas,NaN,48.0,666618,9.071,100
656090,2020-03-03,US,1,Texas,NaN,48.0,677068,9.062,99
656091,2020-03-04,US,1,Texas,NaN,48.0,693767,8.571,94
656092,2020-03-05,US,1,Texas,NaN,48.0,665192,9.828,108


# Filter for California State

In [10]:
california_socialdistancing_data = socialdistancing_data[socialdistancing_data['admin1'] == 'California']
california_socialdistancing_data.head()

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
49681,2020-03-01,US,1,California,NaN,6.0,451282,4.122,60
49682,2020-03-02,US,1,California,NaN,6.0,457175,6.725,98
49683,2020-03-03,US,1,California,NaN,6.0,472598,6.842,100
49684,2020-03-04,US,1,California,NaN,6.0,481255,6.827,99
49685,2020-03-05,US,1,California,NaN,6.0,458159,7.270,106


In [18]:
california_socialdistancing_data.tail()

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
66210,2021-01-03,US,2,California,Yuba County,6115.0,1047,0.161,1
66211,2021-01-04,US,2,California,Yuba County,6115.0,822,0.074,0
66212,2021-01-05,US,2,California,Yuba County,6115.0,839,0.574,6
66213,2021-01-06,US,2,California,Yuba County,6115.0,1151,1.761,18
66214,2021-01-07,US,2,California,Yuba County,6115.0,915,0.082,0


# Filter for New York State

In [11]:
newyork_socialdistancing_data = socialdistancing_data[socialdistancing_data['admin1'] == 'New York']
newyork_socialdistancing_data.head()

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
475777,2020-03-01,US,1,New York,NaN,36.0,261993,2.673,51
475778,2020-03-02,US,1,New York,NaN,36.0,274423,5.176,100
475779,2020-03-03,US,1,New York,NaN,36.0,283364,5.085,98
475780,2020-03-04,US,1,New York,NaN,36.0,294023,5.144,99
475781,2020-03-05,US,1,New York,NaN,36.0,277703,5.479,105


In [17]:
newyork_socialdistancing_data.tail()

,date,country_code,admin_level,admin1,admin2,fips,samples,m50,m50_index
494496,2021-01-03,US,2,New York,Yates County,36123.0,618,0.758,10
494497,2021-01-04,US,2,New York,Yates County,36123.0,579,2.325,33
494498,2021-01-05,US,2,New York,Yates County,36123.0,563,1.719,24
494499,2021-01-06,US,2,New York,Yates County,36123.0,668,2.734,39
494500,2021-01-07,US,2,New York,Yates County,36123.0,595,1.028,14


In [16]:
print(newyork_socialdistancing_data.shape)
print(california_socialdistancing_data.shape)
texas_socialdistancing_data.shape

(18724, 9)
(16534, 9)


(63301, 9)

# Store file to Google drive

In [8]:
!ls '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021'

 CA-CovidDec26.csv		 NY-CovidDec26.csv
 computed_CA_mobility_data.csv	'Policy_Map_(Sized)_Full_Data_data.csv'
 election_results_2021.csv


# Store files to Google drive

In [9]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
filename = 'TX_SocialDistancingData'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
texas_socialdistancing_data.to_csv(location+filename,index=False)

TX_SocialDistancingDataJan10.csv


In [12]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
filename = 'CA_SocialDistancingData'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
california_socialdistancing_data.to_csv(location+filename,index=False)

CA_SocialDistancingDataJan10.csv


In [13]:
from datetime import datetime
location = '/content/drive/Shared drives/CMPE 295- Master Project/projectdata-2021/'
filename = 'NY_SocialDistancingData'+datetime.now().strftime("%b%d")+'.csv' #%Y%m%d
print(filename)
newyork_socialdistancing_data.to_csv(location+filename,index=False)

NY_SocialDistancingDataJan10.csv
